In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['LM(한국어)', 'QA']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    questions = dict()
    answers = dict()
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        while True:
            line = source_data.readline()
            if not line:
                break
            line = json.loads(line)    
            #### data preprocess
            if 'text' in line:
                #### LM(한국어)
                # data = {'text': None}
                ## preprocess data from line
                data = line
                ## preprocess data from line end
                task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
                # print(json.dumps(data, indent=4, ensure_ascii=False))
                #### LM(한국어) end
                continue
            disease_category = line['disease_category']
            disease_name = line['disease_name']['kor']
            intention = line['intention']
            qa_key = '/'.join([disease_category, disease_name, intention])
            if 'question' in line:    
                question = line['question']
                if qa_key not in questions:
                    questions[qa_key] = []
                questions[qa_key].append(question)
            elif 'answer' in line:
                answer = line['answer']
                answer = '\n'.join([answer['intro'], answer['body'], answer['conclusion']])
                if qa_key not in answers:
                    answers[qa_key] = []
                answers[qa_key].append(answer)
            #### data preprocess end 
    
    for qa_key in questions:
        if qa_key not in answers:
            continue
        #### QA
        data = {'input': None, 'output': None}
        ## preprocess data from line
        data['input'] = questions[qa_key]
        data['output'] = answers[qa_key]
        ## preprocess data from line end
        task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
        # print(json.dumps(data, indent=4, ensure_ascii=False))
        #### QA end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 1/1 [00:02<00:00,  2.69s/it]
